# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 00 Game Info

In [ ]:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)

# 01 HYPER PARAMS

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 02 Env+State

In [ ]:
import os

# 파일 복사 명령어 실행
os.system('cp "/content/drive/My Drive/Colab Notebooks/공통 environment+state.ipynb" "/content/"')

0

In [ ]:
import nbformat

notebook_path = "/content/공통 environment+state.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    notebook_content = nbformat.read(f, as_version=4)

# 각 코드 셀 출력 및 실행
for cell in notebook_content.cells:
    if cell.cell_type == "code":
        print(f"실행 중인 코드:\n{cell.source}\n{'='*40}")
        exec(cell.source)

실행 중인 코드:
import torch
import torch.nn as nn
import numpy as np
실행 중인 코드:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)
실행 중인 코드:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
실행 중인 코드:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.n ** 2
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win': 1, 'lose': -1, 'draw': 0} #결과에 따른 보상 정의
        #승리 1점 / 패배 -1점 / 무승부 0점


    def step(self, present_state, action_idx): #현재 상태에서 주어진 행동에 따라 게임 진행 / 행동 결과를 계산
        """
        present_state에 대해 action_idx의 행동에 따라 게임을 한 턴 진행시키고
        next_state, is_done, is_lose를 반환한다.
        """
        #action_idx: 플레이어가 선택한 행동의 인덱스 / 보드 칸
        #현재 상태에서 행동하는 행동을 수행하여 다음 상태를 계산
        next_state = present_state.next(action_idx)
        is_done, is_lose = ne

# 03 MinMax
### 틱택토 상태 평가 신경망

In [ ]:
class TicTacToeEvaluator(nn.Module):
#입력: 3채널 보드 상태
#출력: [-1, 1] (1=승리, -1=패배, 0=무승부/균형 상태)
    def __init__(self, input_shape, hidden_units=128):
        super().__init__()
        self.fc1 = nn.Linear(np.prod(input_shape), hidden_units)
        self.fc2 = nn.Linear(hidden_units, hidden_units)
        self.fc3 = nn.Linear(hidden_units, 1)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()  # 출력 범위 제한 [-1, 1]


    def forward(self, state_tensor):
        x = state_tensor.view(state_tensor.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.tanh(self.fc3(x))

In [ ]:
class MinMaxAgent: #Minimax 알고리즘을 이용해 최적의 행동을 선택하는 에이전트
    def __init__(self, evaluator, environment, depth=3):
        self.evaluator = evaluator
        self.environment = environment
        self.depth = depth


    def get_action(self, state): #현재 상태에서 minmax 탐색을 수행하고 최적의 행동을 반환
        _, best_action = self._minmax(state, self.depth, True)
        # 예외 처리 추가: 가능한 행동이 없을 경우 랜덤 선택
        if best_action is None:
            legal_actions = np.where(state.get_legal_actions() != 0)[0]
            if len(legal_actions) > 0:
                best_action = np.random.choice(legal_actions)  # 가능한 행동 중 랜덤 선택
            else:
                print("[WARNING] No valid actions available. Returning None.")
                return None

        return best_action


    def _minmax(self, state, depth, is_maximizing_player): #Minimax 알고리즘의 재귀 구현
        legal_actions = np.where(state.get_legal_actions() != 0)[0]

        # 게임 종료 상태이거나 탐색 깊이에 도달한 경우
        if len(legal_actions) == 0 or depth == 0:
            # 종료 상태에서 공통 보상 체계 활용
            if state.check_done()[0]:
                return state.get_reward(state), None
            # 그렇지 않다면 신경망 평가값 반환
            return self._evaluate_state(state), None

        # 가능한 행동 목록 가져오기
        if is_maximizing_player: #현재 플레이어가 최적의 행동을 하도록 max_eval 사용
            max_eval = -float('inf')
            best_action = None

            for action in legal_actions:
                next_state = state.next(action)
                eval_value, _ = self._minmax(next_state, depth - 1, False)
                if eval_value > max_eval:
                    max_eval = eval_value
                    best_action = action

            return max_eval, best_action

        else:
            min_eval = float('inf')
            best_action = None

            for action in legal_actions:
                next_state = state.next(action)
                eval_value, _ = self._minmax(next_state, depth - 1, True)
                if eval_value < min_eval:
                    min_eval = eval_value
                    best_action = action

            return min_eval, best_action


    def _evaluate_state(self, state): #현재 상태를 신경망을 이용해 평가
        board_tensor = torch.tensor(
            state.state - state.enemy_state, dtype=torch.float32
        ).unsqueeze(0)
        with torch.no_grad():
            value = self.evaluator(board_tensor).item()
        return value #[-1,1] / 승리 또는 패배 가능성 반영

## Test Code

In [ ]:
DEVICE = "cpu"  # GPU를 사용하려면 "cuda"로 변경

In [ ]:
# TicTacToeEvaluator 초기화
evaluator = TicTacToeEvaluator(input_shape=(3, 3), hidden_units=128).to(DEVICE)
optimizer = torch.optim.Adam(evaluator.parameters(), lr=0.005)
criterion = nn.MSELoss()

# TicTacToeEvaluator 학습을 위한 데이터 생성
train_states = []
train_labels = []

for _ in range(5000):  # 5000개의 학습 데이터 생성
    board_state = torch.randint(-1, 2, (3, 3), dtype=torch.float32)  # 무작위 보드 상태
    label = torch.tensor([[torch.sum(board_state).item() * 0.1]], dtype=torch.float32)  # 단순한 가치 평가
    train_states.append(board_state.view(-1))
    train_labels.append(label)

train_states = torch.stack(train_states).to(DEVICE)
train_labels = torch.stack(train_labels).to(DEVICE)

In [ ]:
# 신경망 학습
print("[DEBUG] Starting TicTacToeEvaluator training...")
for epoch in range(100):
    optimizer.zero_grad()

    predictions = evaluator(train_states)
    loss = criterion(predictions, train_labels)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"[DEBUG] Epoch {epoch}, Loss: {loss.item():.4f}")

print("[DEBUG] TicTacToeEvaluator training completed!")

[DEBUG] Starting TicTacToeEvaluator training...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5000, 1, 1])) that is different to the input size (torch.Size([5000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[DEBUG] Epoch 0, Loss: 0.0659
[DEBUG] Epoch 10, Loss: 0.0613
[DEBUG] Epoch 20, Loss: 0.0613
[DEBUG] Epoch 30, Loss: 0.0611
[DEBUG] Epoch 40, Loss: 0.0611
[DEBUG] Epoch 50, Loss: 0.0610
[DEBUG] Epoch 60, Loss: 0.0610
[DEBUG] Epoch 70, Loss: 0.0610
[DEBUG] Epoch 80, Loss: 0.0610
[DEBUG] Epoch 90, Loss: 0.0610
[DEBUG] TicTacToeEvaluator training completed!


In [ ]:
# MinMaxAgent 생성
environment = Environment()
minmax_agent = MinMaxAgent(evaluator, environment, depth=3)

In [ ]:
# 테스트 1: 초기 상태에서 가능한 행동 확인
initial_state = State()
print("초기 상태 보드:")
initial_state.render(initial_state)

# Minimax 알고리즘이 선택한 행동 확인
best_action = minmax_agent.get_action(initial_state)
print("Minmax가 선택한 최적의 행동:", best_action)

초기 상태 보드:
Minmax가 선택한 최적의 행동: 1


In [ ]:
# 테스트 2: 특정 상태에서 평가
custom_state = State()
custom_state = custom_state.next(3)  # 첫 번째 플레이어가 3번 칸에 놓음
custom_state = custom_state.next(5)  # 두 번째 플레이어가 5번 칸에 놓음
custom_state = custom_state.next(1)  # 첫 번째 플레이어가 1번 칸에 놓음
custom_state = custom_state.next(2)  # 두 번째 플레이어가 2번 칸에 놓음

print("테스트 상태 보드:")
custom_state.render(custom_state)

# Minmax 알고리즘이 선택한 행동 확인
best_action_test = minmax_agent.get_action(custom_state)
print("테스트 상태에서 Minmax 선택 행동:", best_action_test)

테스트 상태 보드:
테스트 상태에서 Minmax 선택 행동: 4
